In [46]:
# Initial imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Dropout, Flatten
import hvplot.pandas


In [47]:
# imports for creating dataframe
from utils_laramie import get_df, get_all_raw_data

# imports for getting weekly range
from utils_laramie import calc_weekly_range

#imports for grouping data into weekly windows
from utils_laramie import grp_y_wk_d, drop_off_weeks

#shape data
from utils_laramie import get_X_y

In [84]:
# get the cleaned data 

df = (get_df(get_all_raw_data()))
df = calc_weekly_range(df)
df = grp_y_wk_d(df)
df = drop_off_weeks(df)
df.tail(10)

,SPY_open,SPY_high,SPY_low,SPY_close,SPY_volume,SPY_trade_count,SPY_vwap,TR,VIX_open,VIX_high,VIX_low,VIX_close,weekly_range
1045,431.58,432.3018,419.36,419.43,131262026.0,1390460.0,424.067609,12.9418,35.88,36.55,32.59,36.45,0.0000
1046,419.87,427.2100,415.12,416.25,158890009.0,1864071.0,419.755053,12.0900,36.19,37.52,32.78,35.13,0.0000
1047,425.16,429.5100,422.83,427.33,110176608.0,1079760.0,426.138376,13.2600,33.74,34.12,31.39,32.45,0.0000
1048,422.42,426.4300,420.44,425.48,91933914.0,891241.0,423.871044,6.8900,33.03,34.03,30.23,30.23,0.0000
1049,428.20,428.7700,419.53,420.07,90803923.0,809145.0,424.040193,9.2400,30.43,31.04,28.84,30.75,17.1818
1050,420.98,424.5500,415.79,417.00,91251505.0,858504.0,419.220077,8.7600,31.03,33.18,30.06,31.77,0.0000
1051,419.66,426.8400,418.42,426.17,104219651.0,920659.0,422.752423,9.8400,33.13,33.83,29.57,29.83,0.0000
1052,429.94,435.6800,424.80,435.55,138130298.0,1344164.0,431.339744,10.8800,29.02,29.80,26.29,26.67,0.0000
1053,433.70,441.0700,433.19,441.07,100157174.0,784018.0,437.706981,7.8800,26.51,27.47,25.25,25.67,0.0000
1054,437.81,444.8600,437.22,444.31,102327793.0,790235.0,441.458916,7.6400,26.36,26.82,23.85,23.87,29.0700


## 3D LSTM Functions
---

In [83]:
# # Set the random seed for reproducibility
# # Note: This is used for model prototyping, but it is good practice to comment this out and run multiple experiments to evaluate your model.
# from numpy.random import seed

# seed(1)
# from tensorflow import random

# random.set_seed(2)

In [63]:
def window_data(df, window, feature_col_1, feature_col_2, target_col):
    '''
    This function accepts the column number for the features (X) and the target (y).
    It chunks the data up with a rolling window of X(t )- window to predict y(t).
    It returns two numpy arrays of X and y.
    '''
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_1:feature_col_2]
        target = df.iloc[(i + window), target_col]
        y.append(target)
        X.append(features)
    return np.array(X), np.array(y).reshape(-1, 1)


In [64]:
X,y = window_data(df,5,0,7,7)
print(X[0:5])
print(y[-5:])


[[[2.10600000e+02 2.11000000e+02 2.08230000e+02 2.08540000e+02
   1.08069059e+08 3.67013000e+05 2.09563055e+02]
  [2.08900000e+02 2.09150000e+02 2.04751100e+02 2.05580000e+02
   1.66224154e+08 5.46768000e+05 2.06878936e+02]
  [2.06100000e+02 2.09970000e+02 2.05930000e+02 2.09660000e+02
   1.92878747e+08 5.56731000e+05 2.08178631e+02]
  [2.09200000e+02 2.09729500e+02 2.07200000e+02 2.08270000e+02
   1.02027111e+08 3.74705000e+05 2.08276128e+02]
  [2.06480000e+02 2.08289000e+02 2.05780000e+02 2.06990000e+02
   1.03372367e+08 3.87782000e+05 2.06966276e+02]]

 [[2.08900000e+02 2.09150000e+02 2.04751100e+02 2.05580000e+02
   1.66224154e+08 5.46768000e+05 2.06878936e+02]
  [2.06100000e+02 2.09970000e+02 2.05930000e+02 2.09660000e+02
   1.92878747e+08 5.56731000e+05 2.08178631e+02]
  [2.09200000e+02 2.09729500e+02 2.07200000e+02 2.08270000e+02
   1.02027111e+08 3.74705000e+05 2.08276128e+02]
  [2.06480000e+02 2.08289000e+02 2.05780000e+02 2.06990000e+02
   1.03372367e+08 3.87782000e+05 2.0696

In [65]:
def data_splited_scaled(df, window, feature_col_1,feature_col_2, target_col):
    '''
    This function splits X and y into training and testing sets, scales the data with MinMaxScaler and reshapes features data for the 3 dimentional LSTM model .
    '''  
    X, y = window_data(df, window, feature_col_1,feature_col_2, target_col)
    # Use 70% of the data for training and the remainder for testing
    split = int(0.7 * len(X))
    X_train = X[: split]
    X_test = X[split:]
    y_train = y[: split]
    y_test = y[split:]

    # Create a MinMaxScaler object
    scaler = MinMaxScaler()

    # Fit the MinMaxScaler object with the training feature data X_train
    scaler.fit(X_train.ravel().reshape(-1,1))

    # Scale the features training and testing sets
    X_train_scaled= scaler.transform(X_train.ravel().reshape(-1,1))
    X_test_scaled = scaler.transform(X_test.ravel().reshape(-1,1))

    # Fit the MinMaxScaler object with the training target data y_train
    scaler.fit(y_train)

    # Scale the target training and testing sets
    y_train_scaled = scaler.transform(y_train)
    y_test_scaled = scaler.transform(y_test)

    # Reshape the features for the model
    feature_num = feature_col_2 - feature_col_1
    X_train_scaled = X_train_scaled.reshape((X_train.shape[0], X_train.shape[1], feature_num))
    X_test_scaled = X_test_scaled.reshape((X_test.shape[0], X_test.shape[1], feature_num))

    return X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, scaler

In [66]:
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, scaler = data_splited_scaled(df,5,0,7,7)
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train_scaled.shape)
print(y_test_scaled.shape)
print(X_train_scaled[0:5])

(1110, 5, 7)
(477, 5, 7)
(1110, 1)
(477, 1)
[[[7.50044037e-08 7.60186620e-08 6.89949230e-08 6.97809732e-08
   2.74024400e-01 9.30155981e-04 7.23750784e-08]
  [7.06938057e-08 7.13277172e-08 6.01736648e-08 6.22754616e-08
   4.21485124e-01 1.38595100e-03 6.55691032e-08]
  [6.35939974e-08 7.34069468e-08 6.31629376e-08 7.26208966e-08
   4.89071732e-01 1.41121364e-03 6.88646695e-08]
  [7.14544995e-08 7.27971240e-08 6.63832078e-08 6.90963489e-08
   2.58704160e-01 9.49660168e-04 6.91118873e-08]
  [6.45575428e-08 6.91445261e-08 6.27825908e-08 6.58507222e-08
   2.62115252e-01 9.82818809e-04 6.57905666e-08]]

 [[7.06938057e-08 7.13277172e-08 6.01736648e-08 6.22754616e-08
   4.21485124e-01 1.38595100e-03 6.55691032e-08]
  [6.35939974e-08 7.34069468e-08 6.31629376e-08 7.26208966e-08
   4.89071732e-01 1.41121364e-03 6.88646695e-08]
  [7.14544995e-08 7.27971240e-08 6.63832078e-08 6.90963489e-08
   2.58704160e-01 9.49660168e-04 6.91118873e-08]
  [6.45575428e-08 6.91445261e-08 6.27825908e-08 6.58507222

In [67]:
def lstm_model(df, window, feature_col_1, feature_col_2, target_col, number_units):
    '''
    This function builds and trains a 3-layer LSTM model to fit the 3 dimentional data.
    '''
    X_train_scaled, _, y_train_scaled, _ ,_= data_splited_scaled(df, window,feature_col_1, feature_col_2, target_col)

    # Define the LSTM RNN model.
    lstm_model = Sequential()

    dropout_fraction = 0.2
    # calculate
    X_train_scaled, _, _, _ ,_= data_splited_scaled(df, window,feature_col_1, feature_col_2, target_col)
    # Layer 1
    feature_num = feature_col_2 - feature_col_1
    lstm_model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train_scaled.shape[1], feature_num))
    )
    lstm_model.add(Dropout(dropout_fraction))
    # Layer 2
    lstm_model.add(LSTM(units=number_units, return_sequences=True))
    lstm_model.add(Dropout(dropout_fraction))
    # Layer 3
    lstm_model.add(LSTM(units=number_units))
    lstm_model.add(Dropout(dropout_fraction))
    # Output layer
    lstm_model.add(Dense(1))

    # Compile the lstm_model
    lstm_model.compile(optimizer="adam", loss="mean_squared_error")

    # Train the lstm_model
    lstm_model.fit(X_train_scaled, y_train_scaled, epochs=10, shuffle=False, batch_size=1, verbose=1)
    return lstm_model
    

In [68]:
lstm_model(df,5,0,7,7,64)

Epoch 1/10
1110/1110 [==============================] - 8s 3ms/step - loss: 0.0051
Epoch 2/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0061
Epoch 3/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0063
Epoch 4/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0059
Epoch 5/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0060
Epoch 6/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0058
Epoch 7/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0061
Epoch 8/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0057
Epoch 9/10
1110/1110 [==============================] - 4s 4ms/step - loss: 0.0056
Epoch 10/10
1110/1110 [==============================] - 4s 4ms/step - loss: 0.0053


In [69]:
def lstm_evaluation(df, window,feature_col_1, feature_col_2, target_col, number_units):
    '''
    This function evaluates the 3d LSTM model
    '''
    _, X_test_scaled, _, y_test_scaled,_ =data_splited_scaled(df, window, feature_col_1, feature_col_2, target_col)
    model = lstm_model(df, window, feature_col_1, feature_col_2, target_col, number_units)
    score = model.evaluate(X_test_scaled, y_test_scaled,verbose=0)
    return score

In [70]:
lstm_evaluation(df,5,0,7,7,64)

Epoch 1/10
1110/1110 [==============================] - 8s 4ms/step - loss: 0.0053
Epoch 2/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0063
Epoch 3/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0064
Epoch 4/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0062
Epoch 5/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0059
Epoch 6/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0061
Epoch 7/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0059
Epoch 8/10
1110/1110 [==============================] - 4s 4ms/step - loss: 0.0059
Epoch 9/10
1110/1110 [==============================] - 4s 4ms/step - loss: 0.0057
Epoch 10/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0077


0.022871877998113632

In [72]:
def lstm_prediction(df, window, feature_col_1, feature_col_2, target_col, number_units):
    '''
    This function predicts y values and recover the original prices, and then creates a dataframe of Acural and Predicted values of y
    '''
    _, X_test_scaled, _, y_test_scaled,scaler =data_splited_scaled(df, window, feature_col_1, feature_col_2, target_col)
    model= lstm_model(df, window, feature_col_1, feature_col_2, target_col, number_units)
    y_predicted = model.predict(X_test_scaled)

    # Recover the original prices instead of the scaled version
    predicted_prices = scaler.inverse_transform(y_predicted)
    actual_prices = scaler.inverse_transform(y_test_scaled.reshape(-1, 1))

    prediction_df = pd.DataFrame({
        "actual":actual_prices.ravel(),
        "predicted":predicted_prices.ravel(),
    })
    
    return prediction_df

In [75]:
prediction_df = lstm_prediction(df,5,0,7,7,64)
prediction_df

Epoch 1/10
1110/1110 [==============================] - 8s 4ms/step - loss: 0.0049
Epoch 2/10
1110/1110 [==============================] - 4s 4ms/step - loss: 0.0061
Epoch 3/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0062
Epoch 4/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0062
Epoch 5/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0061
Epoch 6/10
1110/1110 [==============================] - 4s 3ms/step - loss: 0.0059
Epoch 7/10
1110/1110 [==============================] - 4s 4ms/step - loss: 0.0060
Epoch 8/10
1110/1110 [==============================] - 4s 4ms/step - loss: 0.0057
Epoch 9/10
1110/1110 [==============================] - 4s 4ms/step - loss: 0.0060
Epoch 10/10
15/15 [==============================] - 1s 3ms/step


,actual,predicted
0,5.270,6.427706
1,4.120,6.658632
2,7.720,6.685344
3,8.225,6.702477
4,12.120,6.644905
...,...,...
472,5.880,6.317099
473,6.700,6.459174
474,4.550,6.518028
475,5.850,6.658626


## 4D LSTM Functions

In [85]:
def window_data(df, window, chunk_rows, feature_col_1, feature_col_2, target_col):
    X_list = df.iloc[:,feature_col_1:feature_col_2].values.tolist()
    X_chunks=[X_list[i:i + chunk_rows] for i in range(0, len(X_list), chunk_rows)]
    X = [X_chunks[i:i+window] for i in range (len(X_chunks)-window)]
    y_list=df.iloc[:,target_col].values.tolist()
    y_chunks = [y_list[i + chunk_rows-1] for i in range(0, len(y_list), chunk_rows)]
    y = [y_chunks[i+window] for i in range(len(y_chunks)-window)]
    return np.array(X), np.array(y)

In [88]:
X, y =window_data(df,3,5,0,12,12)
print(X[0:5])
print(y[-5:])

[[[[2.09200000e+02 2.09729500e+02 2.07200000e+02 2.08270000e+02
    1.02027111e+08 3.74705000e+05 2.08276128e+02 2.52950000e+00
    1.56500000e+01 1.71800000e+01 1.55800000e+01 1.58400000e+01]
   [2.06480000e+02 2.08289000e+02 2.05780000e+02 2.06990000e+02
    1.03372367e+08 3.87782000e+05 2.06966276e+02 2.50900000e+00
    1.76900000e+01 1.83300000e+01 1.65200000e+01 1.76000000e+01]
   [2.06200000e+02 2.08680000e+02 2.04180000e+02 2.05330000e+02
    1.62401537e+08 5.86210000e+05 2.06034646e+02 4.50000000e+00
    1.80500000e+01 2.01300000e+01 1.57200000e+01 1.96100000e+01]
   [2.05440000e+02 2.07430000e+02 2.05140000e+02 2.05860000e+02
    1.16128858e+08 4.04992000e+05 2.06102975e+02 2.29000000e+00
    1.92500000e+01 1.97200000e+01 1.81300000e+01 1.93400000e+01]
   [2.03380000e+02 2.04140000e+02 2.01510000e+02 2.01880000e+02
    2.11173305e+08 6.69924000e+05 2.03150102e+02 4.35000000e+00
    2.13600000e+01 2.52700000e+01 2.08800000e+01 2.43900000e+01]]

  [[1.93050000e+02 1.93410000e+02

In [89]:

def data_splited_scaled(df, window, chunk_rows, feature_col_1,feature_col_2, target_col):
    '''
    This function splits X and y into training and testing sets, scales the data with MinMaxScaler and reshapes features data for the LSTM model .
    '''
    X, y = window_data(df, window, chunk_rows, feature_col_1,feature_col_2, target_col)
    # Use 70% of the data for training and the remainder for testing
    split = int(0.7 * len(X))
    X_train = X[: split]
    X_test = X[split:]
    y_train = y[: split]
    y_test = y[split:]

    # Create a MinMaxScaler object
    scaler = MinMaxScaler()

    # Fit the MinMaxScaler object with the training feature data X_train
    X_scaler = scaler.fit(X_train.ravel().reshape(-1,1))

    # Scale the features training and testing sets
    X_train_scaled= X_scaler.transform(X_train.ravel().reshape(-1,1))
    X_test_scaled = X_scaler.transform(X_test.ravel().reshape(-1,1))

    # Fit the MinMaxScaler object with the training target data y_train
    y_scaler = scaler.fit(y_train.ravel().reshape(-1,1))

    # Scale the target training and testing sets
    y_train_scaled = y_scaler.transform(y_train.ravel().reshape(-1,1))
    y_test_scaled = y_scaler.transform(y_test.ravel().reshape(-1,1))

    # Reshape the features for the model
    feature_num = feature_col_2 - feature_col_1
    X_train_scaled = X_train_scaled.reshape((X_train.shape[0], X_train.shape[1],X_train.shape[2], feature_num))
    X_test_scaled = X_test_scaled.reshape((X_test.shape[0], X_test.shape[1],X_train.shape[2], feature_num))

    return X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, scaler


In [91]:
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, scaler=data_splited_scaled(df,3,5,0,12,12)
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train_scaled.shape)
print(y_test_scaled.shape)
print(X_train_scaled[0:5])

(145, 3, 5, 12)
(63, 3, 5, 12)
(145, 1)
(63, 1)
[[[[5.29315825e-07 5.30658449e-07 5.24244535e-07 5.26957675e-07
    2.58704499e-01 9.50117595e-04 5.26973214e-07 5.27287309e-09
    3.85417990e-08 4.24213353e-08 3.83643038e-08 3.90235715e-08]
   [5.22418871e-07 5.27005852e-07 5.20643920e-07 5.23712050e-07
    2.62115590e-01 9.83276220e-04 5.23651894e-07 5.22089237e-09
    4.37145141e-08 4.53373267e-08 4.07478098e-08 4.34863061e-08]
   [5.21708891e-07 5.27997289e-07 5.16586888e-07 5.19502880e-07
    4.11792590e-01 1.48641912e-03 5.21289612e-07 1.02693609e-08
    4.46273462e-08 4.99014871e-08 3.87192941e-08 4.85829519e-08]
   [5.19781801e-07 5.24827734e-07 5.19021107e-07 5.20846772e-07
    2.94461519e-01 1.02691466e-03 5.21462870e-07 4.66558619e-09
    4.76701198e-08 4.88618728e-08 4.48301978e-08 4.78983278e-08]
   [5.14558373e-07 5.16485463e-07 5.09816717e-07 5.10754906e-07
    5.35460464e-01 1.69868808e-03 5.13975433e-07 9.88901421e-09
    5.30203300e-08 6.29347007e-08 5.18032206e-08 6.0

In [100]:

def lstm_model(df, window, chunk_rows, feature_col_1, feature_col_2, target_col, number_units):
    '''
    This function builds and trains a 3-layer LSTM model

    '''

    X_train_scaled, _, y_train_scaled, _ ,_= data_splited_scaled(df, window, chunk_rows, feature_col_1,feature_col_2, target_col)

    # Define the LSTM RNN model.
    lstm_model = Sequential()

    dropout_fraction = 0.2
    # calculate
    X_train_scaled, _, _, _ ,_= data_splited_scaled(df, window, chunk_rows, feature_col_1, feature_col_2, target_col)

    
    # use TimeDistributed to get proper input shape of X_train
    # Layer 1
    lstm_model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(window, chunk_rows*(feature_col_2-feature_col_1))
    ))
    lstm_model.add(Dropout(dropout_fraction))
    # Layer 2
    lstm_model.add(LSTM(units=number_units, return_sequences=True))
    lstm_model.add(Dropout(dropout_fraction))
    # Layer 3
    lstm_model.add(LSTM(units=number_units))
    lstm_model.add(Dropout(dropout_fraction))
    # Output layer
    lstm_model.add(Dense(1))

    # Compile the lstm_model
    lstm_model.compile(optimizer="adam", loss="mean_squared_error")

    # Train the lstm_model
    lstm_model.fit(X_train_scaled.reshape(X_train_scaled.shape[0],X_train_scaled.shape[1],-1), y_train_scaled, epochs=10, shuffle=False, batch_size=1, verbose=1)

    return lstm_model


In [101]:
lstm_model(df,3,5,0,12,12,64)

Epoch 1/10
145/145 [==============================] - 4s 3ms/step - loss: 0.0164
Epoch 2/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0182
Epoch 3/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0187
Epoch 4/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0189
Epoch 5/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0188
Epoch 6/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0191
Epoch 7/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0187
Epoch 8/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0189
Epoch 9/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0184
Epoch 10/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0187


In [105]:

def lstm_evaluation(df, window,chunk_rows, feature_col_1, feature_col_2, target_col, number_units):
    '''
    This function evaluates the LSTM model
    '''
    _, X_test_scaled, _, y_test_scaled,_ =data_splited_scaled(df, window,chunk_rows, feature_col_1, feature_col_2, target_col)
    model = lstm_model(df, window, chunk_rows,feature_col_1, feature_col_2, target_col, number_units)
    score = model.evaluate(X_test_scaled.reshape(X_test_scaled.shape[0],X_test_scaled.shape[1],-1), y_test_scaled,verbose=0)
    return score
    

In [106]:
lstm_evaluation(df,3,5,0,12,12,64)

Epoch 1/10
145/145 [==============================] - 4s 3ms/step - loss: 0.0155
Epoch 2/10
145/145 [==============================] - 1s 4ms/step - loss: 0.0183
Epoch 3/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0188
Epoch 4/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0186
Epoch 5/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0192
Epoch 6/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0187
Epoch 7/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0192
Epoch 8/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0191
Epoch 9/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0181
Epoch 10/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0187


0.018896009773015976

In [116]:
def lstm_prediction(df, window, chunk_rows, feature_col_1, feature_col_2,target_col, number_units):
    '''
    This function predicts y values and recover the original prices, and then creates a dataframe of Acural and Predicted values of y
    '''
    _, X_test_scaled, _, y_test_scaled,scaler =data_splited_scaled(df, window,chunk_rows, feature_col_1,feature_col_2, target_col)
    model= lstm_model(df, window, chunk_rows, feature_col_1, feature_col_2, target_col, number_units)
    y_predicted = model.predict(X_test_scaled.reshape(X_test_scaled.shape[0],X_test_scaled.shape[1],-1))

    # Recover the original prices instead of the scaled version
    predicted = scaler.inverse_transform(y_predicted)
    actual = scaler.inverse_transform(y_test_scaled.reshape(-1, 1))

    prediction_df = pd.DataFrame({
        "actual":actual.ravel(),
        "predicted":predicted.ravel(),
    })

    return prediction_df


In [117]:
prediction_df = lstm_prediction(df,3,5,0,12,12,64)

Epoch 1/10
145/145 [==============================] - 5s 3ms/step - loss: 0.0160
Epoch 2/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0187
Epoch 3/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0190
Epoch 4/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0191
Epoch 5/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0185
Epoch 6/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0190
Epoch 7/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0193
Epoch 8/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0191
Epoch 9/10
145/145 [==============================] - 0s 3ms/step - loss: 0.0186
Epoch 10/10
2/2 [==============================] - 1s 3ms/step


In [118]:
prediction_df

,actual,predicted
0,5.9200,9.001218
1,18.2150,8.924905
2,24.8100,8.978096
3,18.9000,9.089535
4,15.0800,9.238941
...,...,...
58,18.3100,8.898646
59,18.9400,9.098240
60,13.9000,9.110842
61,17.1818,9.186761


# Ploting Funtions
---

In [24]:
def history_plotting(df, title):

    '''
    This function plots the historical data.
    '''

    return df.hvplot(
                    ylabel = 'Price in $',
                    width= 1000,
                    height=400,
                    title=title)
     

In [26]:
def moving_avgs_plotting(df,ma_column_1, ma_column_2, title):
    '''
    This function plots the overlay of the original prices and moving averages.
    '''
    prices = df['close'].hvplot(line_color='lightgray',
                        ylabel='Price in $',
                        width=1000,
                        height=400,
                        title=title
                        )

    moving_avgs = df[[ma_column_1, ma_column_2]].hvplot(
                            ylabel='Price in $',
                            width=1000,
                            height=400
    )   

    return prices*moving_avgs

    

In [82]:
def predicted_plotting(df,index, ylabel,title):
    '''
    This function plots the actual prices vs. the predicted values.
    '''
    return df[["actual","predicted"]].plot(
                    ylabel= ylabel,
                    title=title
).axvline(index, color='red',label='prediction start')

In [28]:
# def bands_plotting(df, title ):
#     '''
#     This function plots the range of the prices.
#     '''
#     close = df['close'].hvplot(
#         line_color="lightgray",
#         ylabel="Price in $",
#         width=1000,
#         height=400,
#         title=title
#     )

#     upper_band = df['upper'].hvplot(
#         line_color="purple",
#         ylabel = "Price in $",
#         width=1000,
#         height=400,
#     )

#     middle = df['middle'].hvplot(
#         line_color="orange",
#         ylabel = "Price in $",
#         width = 1000,
#         height = 400
#     )


#     lower_band = df['lower'].hvplot(
#         line_color="blue",
#         ylabel = "Price in $",
#         width = 1000,
#         height = 400
#     )

#     # Overlay plots
#     return close * upper_band *middle *lower_band

In [29]:
# def entry_exit_positions(df, title):
#     '''
#     This function plots the entry/exit positions using ranges.
#     '''
#     entry = df[df["signal"] == 1.0]["close"].hvplot.scatter(
#     color="green",
#     marker="^",
#     size=200,
#     legend=False,
#     ylabel="Price in $",
#     title=title,
#     width=1000,
#     height=400,
# )

# # Visualize exit position relative to close price
#     exit = df[df["signal"] == -1.0]["close"].hvplot.scatter(
#     color="red",
#     marker="v",
#     size=200,
#     legend=False,
#     ylabel="Price in $",
#     width=1000,
#     height=400
# )

# # Visualize close price for the investment
#     close = df[["close"]].hvplot(
#     line_color="lightgray",
#     ylabel="Price in $",
#     width=1000,
#     height=400
# )

#     upper = df[["upper"]].hvplot(
#     line_color="purple",
#     ylabel="Price in $",
#     width=1000,
#     height=400
# )

#     middle = df[["middle"]].hvplot(
#     line_color="orange",
#     ylabel="Price in $",
#     width=1000,
#     height=400
# )

#     lower = df[["Lower"]].hvplot(
#     line_color="blue",
#     ylabel="Price in $",
#     width=1000,
#     height=400
# )


# # Overlay plots
#     return close * upper * middle * lower * entry * exit